In [1]:
import torch
import torchvision as tv
import numpy as np
import sys
import os
import random
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
class FusionDataset(torch.utils.data.Dataset):
    def __init__(self, array, transform=None):
        self.array = array
        self.transform = transform
    
    def __getitem__(self, index):
        sample = self.array[index,1:].reshape((636,256,8))
        label = int(self.array[index,0])
        if self.transform:
            sample = self.transform(sample).float()
        return (sample, label)
    
    def __len__(self):
        return self.array.shape[0]

In [3]:
load_model = False

window_size = 20
num_epochs = 10
batch_size = 40
data_folder = 'combined_data'
classes = 8

In [4]:
class MyModel1(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.ModuleDict({
            'lstm': torch.nn.LSTM(636*256,32,batch_first=True),
            'linear': torch.nn.Linear(32,classes),
            'sigmoid': torch.nn.Sigmoid()
        })
    def forward(self,x):
        out = torch.reshape(x,(*(x.size()[0:2]), -1))
        out,_ = self.model['lstm'](out)
        out = self.model['linear'](out)
        out = self.model['sigmoid'](out)
        return out[:,-1,...]
    def loss_function(self,x,y):
        output = self.forward(x)
        loss = torch.nn.CrossEntropyLoss()(output,y)
        return loss

In [5]:
class MyModel2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.ModuleDict({
            'conv1': torch.nn.Conv2d(window_size,window_size,5,stride=2),
            'relu1': torch.nn.ReLU(),
            'conv2': torch.nn.Conv2d(window_size,window_size,5,stride=2),
            'relu2': torch.nn.ReLU(),
            'lstm': torch.nn.LSTM(9516,32,batch_first=True),
            'linear': torch.nn.Linear(32,classes),
            'sigmoid': torch.nn.Sigmoid()
        })
    def forward(self,x):
        out = self.model['conv1'](x)
        out = self.model['relu1'](out)
        out = self.model['conv2'](out)
        out = self.model['relu2'](out)
        #out = torch.reshape(out,(batch_size,window_size,-1))
        out = torch.reshape(out,(*(out.size()[0:2]), -1))
        out,_ = self.model['lstm'](out)
        out = self.model['linear'](out)
        out = self.model['sigmoid'](out)
        return out[:,-1,...]
    def loss_function(self,x,y):
        output = self.forward(x)
        loss = torch.nn.CrossEntropyLoss()(output,y)
        return loss

In [6]:
network = MyModel1()
if load_model:
    network.load_state_dict(torch.load('rnn_network_mod_adamw1.pt'),strict=False)

optimizer = torch.optim.AdamW(network.parameters(), lr=5e-4)

In [7]:
print('loading training data as Numpy and coverting...')
files = [os.path.join(data_folder,f) for f in os.listdir(data_folder) if f.endswith('_train.npy')]
random.shuffle(files)

losses = []
for im in files:
    im_data = np.load(im)
    print(im,im_data.shape)
    dataset = FusionDataset(im_data,transform=tv.transforms.ToTensor())
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True,drop_last=True)
    file_loss = []
    for epoch in range(num_epochs):
        epoch_loss = []
        for batch,(seq,labels) in enumerate(dataloader):
            optimizer.zero_grad()
            loss = network.loss_function(seq,labels)
            loss.backward()
            optimizer.step()
            loss_val = loss.item()
            print('Epoch',epoch,'Batch',batch,'Loss:',loss_val,end='\r')
            epoch_loss.append(loss_val)
        file_loss.append(epoch_loss)
    losses.append(file_loss)
    del im_data
losses = np.array(losses)

loading training data as Numpy and coverting...
combined_data/data_10_train.npy (534, 1302529)
combined_data/data_4_train.npy (535, 1302529)
combined_data/data_18_train.npy (534, 1302529)
combined_data/data_2_train.npy (533, 1302529)
combined_data/data_14_train.npy (534, 1302529)
combined_data/data_11_train.npy (534, 1302529)
combined_data/data_15_train.npy (534, 1302529)
combined_data/data_6_train.npy (531, 1302529)
combined_data/data_5_train.npy (533, 1302529)
combined_data/data_9_train.npy (530, 1302529)
combined_data/data_0_train.npy (536, 1302529)
combined_data/data_12_train.npy (535, 1302529)
combined_data/data_7_train.npy (535, 1302529)
combined_data/data_13_train.npy (535, 1302529)
combined_data/data_1_train.npy (534, 1302529)
combined_data/data_3_train.npy (535, 1302529)
combined_data/data_8_train.npy (536, 1302529)
combined_data/data_16_train.npy (535, 1302529)
combined_data/data_17_train.npy (535, 1302529)
combined_data/data_19_train.npy (535, 1302529)


In [8]:
np.save('train_losses_new5.npy',losses)

In [12]:
print('loading testing data as Numpy and coverting...')
test_files = [os.path.join(data_folder,f) for f in os.listdir(data_folder) if f.endswith('_test.npy')]
random.shuffle(test_files)

labels = []
preds = []
for im in test_files:
    im_data = np.load(im)
    print(im,im_data.shape)
    dataset = FusionDataset(im_data,transform=tv.transforms.ToTensor())
    dataloader = torch.utils.data.DataLoader(dataset)
    for seq,label in dataloader:
        pred = network(seq)
        print(pred)
        pred = torch.argmax(pred)
        labels.append(label.item())
        preds.append(pred.item())

loading testing data as Numpy and coverting...
combined_data/data_16_test.npy (228, 1302529)
tensor([[0.9513, 0.1846, 0.7573, 0.9085, 0.0813, 0.2848, 0.1132, 0.0761]],
       grad_fn=<SelectBackward>)
tensor([[0.9513, 0.1846, 0.7573, 0.9085, 0.0813, 0.2848, 0.1132, 0.0761]],
       grad_fn=<SelectBackward>)
tensor([[0.9513, 0.1846, 0.7573, 0.9085, 0.0813, 0.2848, 0.1132, 0.0761]],
       grad_fn=<SelectBackward>)
tensor([[0.9513, 0.1846, 0.7573, 0.9085, 0.0813, 0.2848, 0.1132, 0.0761]],
       grad_fn=<SelectBackward>)
tensor([[0.9513, 0.1846, 0.7573, 0.9085, 0.0813, 0.2848, 0.1132, 0.0761]],
       grad_fn=<SelectBackward>)
tensor([[0.9513, 0.1846, 0.7573, 0.9085, 0.0813, 0.2848, 0.1132, 0.0761]],
       grad_fn=<SelectBackward>)
tensor([[0.9513, 0.1846, 0.7573, 0.9085, 0.0813, 0.2848, 0.1132, 0.0761]],
       grad_fn=<SelectBackward>)
tensor([[0.9513, 0.1846, 0.7573, 0.9085, 0.0813, 0.2848, 0.1132, 0.0761]],
       grad_fn=<SelectBackward>)
tensor([[0.9513, 0.1846, 0.7573, 0.9085, 0.

KeyboardInterrupt: 

In [10]:
conf = confusion_matrix(labels,preds)
print(conf)

[[784   0  42   0   0   0   0   0]
 [392   0  21   0   0   0   0   0]
 [784   0  42   0   0   0   0   0]
 [784   0  42   0   0   0   0   0]
 [392   0  21   0   0   0   0   0]
 [392   0  21   0   0   0   0   0]
 [392   0  21   0   0   0   0   0]
 [392   0  21   0   0   0   0   0]]


In [11]:
plt.plot(losses.flatten())

In [11]:
torch.save(network.state_dict(),'rnn_network_mod2.pt')